Please download the datasets from here: https://drive.google.com/open?id=1mNEblJS0622w5-2mB6yCyMbu7RbcfENL<br>
This should contain the following:
- __5_min_train__: training data as np array. Shape: (7500, 64, 64, 8)
- __5_min_xval__: validation data (currently unused) as np array. Shape: (1500, 64, 64, 8)
- __5_min_test__: test data (used as visual validation during training) as np array. Shape: (1000, 64, 64, 8)
- __5_min_norms__: list of floats containing the maximum pixel intensity value prior to normalization for each sequence. Shape: (10000,)
- __5_min_long_pred__: test data for sequence prediction as np array. We used it for testing after training. Shape: (1000, 64, 64, 20)
- __5_min_long_pred_norms__: list of floats containing the maximum pixel intensity value prior to normalization for each sequence for the 5_min_long_pred dataset. Shape: (1000,)
- __tgan_1/2/4-1_vx/vy_2000__: optical flow images between the last and second last frames of the input for the first 2000 sequences of the training dataset (__5_min_train__) as np array. The 1/2/4 means the length of the input sequence. We mostly used 2. Shape: (2000, 64, 64, 1)
- __germany__: Not needed. (GPS coordinates of Germany. Used for experimenting before.)

In the datasets the first axis is stands for the sample, the next two for the frame height and width and the last for the channels which is the time axis here.<br>
If data is missing or you cannot acces the drive, please write me an E-Mail: pkicsiny@gmail.com

In [1]:
import src
import keras.backend as K
import os
import numpy as np
import sys
from scipy import ndimage
import pandas as pd
#models folder
sys.path.insert(0, 'C:/Users/pkicsiny/Desktop/TUM/3/ADL4CV/ADL4CV_project/models/')
#data folder
sys.path.insert(0, 'C:/Users/pkicsiny/Desktop/TUM/3/ADL4CV/data')
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0" #"" or "-1" for CPU, "0" for GPU
from tensorflow import keras
from keras.models import load_model
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5141474448704240188
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1508248780
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4631523335830454397
physical_device_desc: "device: 0, name: GeForce GT 740M, pci bus id: 0000:01:00.0, compute capability: 3.5"
]


Some general parameters.

In [2]:
past = 2
name = f"tgan_{past}-1"
batch_size=16
iterations = 2500
#generator trainings in 1 iter
g = 2
#spatial disc trainings in 1 iter
s = 1
#temporal disc trainings in 1 iter
t = 1
#initialize random seed
RND = 777
np.random.seed(RND)
#set this to true if you want to use a pretrained model and load its weights from file
use_loaded = True

## Load dataset

In [ ]:
#past+1 is bc of temporal raining. See second presentation.
train, xval, test = src.load_datasets(past_frames=past+1)

Split data to inputs and ground truth images. Does data augmentation by rotations. Uses the first 2000 samples (bc. of memory issue.)

In [ ]:
gan_train, gan_truth, gan_val, gan_val_truth, gan_test, gan_test_truth = src.split_datasets(
            train[:2000], xval, test, past_frames=past+1, augment=True)

Calculate optical flows between frame t-1 and t.

In [ ]:
#optical flow of the augmented data of the first 2000 training images (8000 images)
#vx, vy = src.optical_flow(gan_train[:,:,:,-2:-1], gan_train[:,:,:,-1:], window_size=4, tau=1e-2, init=1) # (n,:,:,1)

Save optical flows

In [ ]:
#np.savez_compressed(f"{name}_vx_2000",vx) #2000 denotes that they re the flo2 of the first 2000 samples from the training dataset
#np.savez_compressed(f"{name}_vy_2000",vy)

If optical flows are saved, load them

In [ ]:
vx = np.load(sys.path[0]+f"/{name}_vx_2000.npz")["arr_0"]
vy = np.load(sys.path[0]+f"/{name}_vy_2000.npz")["arr_0"]

Preprocess optical flows

In [ ]:
normalized_optical_flow = src.normalize_flows(vx, vy)
flows = np.transpose([[ndimage.median_filter(image[..., ch], 4) for ch in range(2)] for image in normalized_optical_flow], (0, 2, 3, 1))

Make discriminator labels.

In [ ]:
# Adversarial ground truths
real = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

Switch 5% of labels and smooth from 1 to 0.9.

In [ ]:
#real, fake = src.noisy_d_labels(real, fake)

## Build network

Make generator but don't compile.

In [12]:
generator = src.unet((64, 64, past), dropout=0, batchnorm=True, kernel_size=5, feature_mult=1)

Make discriminators and don't compile.

In [13]:
s_discriminator = src.spatial_discriminator(condition_shape=(64, 64, past), dropout = 0.5, batchnorm=True)
t_discriminator = src.temporal_discriminator(dropout = 0.5, batchnorm=True)

Inputs and outputs of the GAN.

In [14]:
frame_t = keras.layers.Input(shape=(64, 64, past))
adv = keras.layers.Input(shape=(64, 64, 1))

In [15]:
generated = generator(frame_t)
s_score = s_discriminator([frame_t, generated])
t_score = t_discriminator([adv, generated])

Loss weights.

In [16]:
loss_weights=[0.1, 1, 1]

Make combined model.

In [17]:
combined = keras.models.Model(inputs=[frame_t, adv], outputs=[generated, s_score, t_score])

Compile discriminators with trainable weights.

In [9]:
s_discriminator.compile(loss=keras.losses.binary_crossentropy,optimizer=keras.optimizers.SGD(),
                      metrics=[keras.metrics.binary_accuracy])
t_discriminator.compile(loss=keras.losses.binary_crossentropy,optimizer=keras.optimizers.SGD(),
                      metrics=[keras.metrics.binary_accuracy])

Freeze discriminator weights and compile combined GAN with only the generator trainable.

In [10]:
s_discriminator.trainable = False
t_discriminator.trainable = False
combined.compile(loss=[src.custom_loss(loss="l1"),
                       keras.losses.binary_crossentropy,
                       keras.losses.binary_crossentropy],
                 optimizer=keras.optimizers.Adam(0.0002, 0.5),
                 loss_weights=loss_weights,
                 metrics=[src.relative_error_tensor, "accuracy", "accuracy"])

***Using L1 loss.***
Tensor("metrics_2/relative_error_tensor/truediv:0", shape=(), dtype=float32)
Tensor("model_1_target:0", shape=(?, ?, ?, ?), dtype=float32)
Tensor("metrics_2/relative_error_tensor_1/truediv:0", shape=(), dtype=float32)
Tensor("model_2_target:0", shape=(?, ?), dtype=float32)
Tensor("metrics_2/relative_error_tensor_2/truediv:0", shape=(), dtype=float32)
Tensor("model_3_target:0", shape=(?, ?), dtype=float32)


If there is a pretrained model load and use its weights. Also set discriminator weights to trainable then recompile model.

In [18]:
if use_loaded:
    combined.load_weights(sys.path[1]+name+"/"+name+"_model.h5")
    generator = combined.layers[1]
    s_discriminator = combined.layers[3]
    s_discriminator.trainable = True
    for l in s_discriminator.layers: l.trainable = True
    t_discriminator = combined.layers[4]
    t_discriminator.trainable = True
    for l in t_discriminator.layers: l.trainable = True
    s_discriminator.compile(loss=keras.losses.binary_crossentropy,optimizer=keras.optimizers.SGD(),
                      metrics=[keras.metrics.binary_accuracy])
    t_discriminator.compile(loss=keras.losses.binary_crossentropy,optimizer=keras.optimizers.SGD(),
                      metrics=[keras.metrics.binary_accuracy])
    s_discriminator.trainable = False
    t_discriminator.trainable = False
    combined.compile(loss=[src.custom_loss(loss="l1"),
                       keras.losses.binary_crossentropy,
                       keras.losses.binary_crossentropy],
                     optimizer=keras.optimizers.Adam(0.0002, 0.5),
                     loss_weights=loss_weights,
                     metrics=[src.relative_error_tensor, "accuracy", "accuracy"])

Log dict. Either append to existing log or start with an empty one.

In [ ]:
#nb_batches = int(gan_train.shape[0] / batch_size)

if use_loaded:
    log = np.load(sys.path[1]+name+"/"+name+"_log.npy").item()
else:
    log = {"g_loss":[],
           "d_loss":[],
           "ds_loss":[],
           "dt_loss":[],
           "g_metric":[],
           "d_metric":[],
           "ds_metric":[],
           "dt_metric":[],
           "d_loss_real":[],
           "d_loss_fake":[],
           "ds_loss_real":[],
           "ds_loss_fake":[],
           "dt_loss_real":[],
           "dt_loss_fake":[],}

## Training

In [ ]:
for it in range(iterations):
#batch
#idx = range(it%nb_batches * batch_size,(it%nb_batches + 1) * batch_size)
#DS create training batch
    s_discriminator.trainable = True
    for tds in range(s):
        idx = np.random.choice(gan_train.shape[0], batch_size, replace=False)
        real_imgs = gan_truth[idx]
        training_batch = gan_train[idx,:,:,1:]
        #predict next frame  
        generated_imgs = generator.predict(training_batch) 
        #train discriminator
        ds_loss_real = s_discriminator.train_on_batch([training_batch, real_imgs], real)
        ds_loss_fake = s_discriminator.train_on_batch([training_batch, generated_imgs], fake)
        #add losses
        ds_loss = 0.5 * np.add(ds_loss_real, ds_loss_fake)
    s_discriminator.trainable = False

#DT create training batch
    t_discriminator.trainable = True
    for tdt in range(t):
        idx = np.random.choice(gan_train.shape[0], batch_size, replace=False)
        real_imgs = gan_truth[idx]
        training_batch = gan_train[idx,:,:,1:]
        
        #predict frame t from frame t-1 and advect
        aux_batch = gan_train[idx,:,:,:-1]
        
        advected_aux_gen = generator.predict(aux_batch)
        advected_aux_truth = training_batch[:,:,:,-1:]
        for i in range(10):
            advected_aux_gen = np.array([src.advect(sample, order=2) for sample in np.concatenate((advected_aux_gen, -flows[idx]), axis=-1)])
            advected_aux_truth = np.array([src.advect(sample, order=2) for sample in np.concatenate((advected_aux_truth, -flows[idx]), axis=-1)])
     
        #predict next frame
        generated_imgs = generator.predict(training_batch) 
        #train discriminator
        dt_loss_real = t_discriminator.train_on_batch([advected_aux_truth, real_imgs], real)
        dt_loss_fake = t_discriminator.train_on_batch([advected_aux_gen, generated_imgs], fake)
        #add losses
        dt_loss = 0.5 * np.add(dt_loss_real, dt_loss_fake)
    t_discriminator.trainable = False
    
#Total discriminator loss
    d_loss = 0.5 * np.add(ds_loss, dt_loss)
    d_loss_real = 0.5 * np.add(ds_loss_real, dt_loss_real)
    d_loss_fake = 0.5 * np.add(ds_loss_fake, dt_loss_fake)
    
    if it%50 == 0 and it%100 != 0:
        src.plot_advections(advected_aux_gen, advected_aux_truth, it)

#Generator
    for tg in range(g):
        idx = np.random.choice(gan_train.shape[0], batch_size, replace=False)
        real_imgs = gan_truth[idx]
        training_batch = gan_train[idx,:,:,1:]
        
        aux_batch = gan_train[idx,:,:,:-1]
        advected = generator.predict(aux_batch)
        for i in range(10):
            advected = np.array([src.advect(sample, order=2) for sample in np.concatenate((advected, -flows[idx]), axis=-1)])
    
        g_loss = combined.train_on_batch([training_batch, advected], [real_imgs, real, real])
    
    log["g_loss"].append(g_loss)
    log["d_loss"].append(d_loss)
    log["ds_loss"].append(ds_loss[0])  
    log["dt_loss"].append(dt_loss[0])
    log["g_metric"].append(g_loss[1])
    log["d_metric"].append(d_loss[1])
    log["ds_metric"].append(ds_loss[1])
    log["dt_metric"].append(dt_loss[1])
    log["d_loss_real"].append(d_loss_real)
    log["d_loss_fake"].append(d_loss_fake)
    log["ds_loss_real"].append(ds_loss_real)
    log["ds_loss_fake"].append(ds_loss_fake)
    log["dt_loss_real"].append(dt_loss_real)
    log["dt_loss_fake"].append(dt_loss_fake)
    
    
    print(f"\033[1m {it} [Ds loss: {ds_loss[0]}, acc.: {100*ds_loss[1]}]\033[0m \n"+
          f"\033[1m {it} [Dt loss: {dt_loss[0]}, acc.: {100*dt_loss[1]}]\033[0m \n"+
          f"\033[1m {it} [G loss: {g_loss[0]}, G obj.: {g_loss[1]}, Gs bce.: {g_loss[2]}, Gt bce.: {g_loss[3]}]\033[0m")
    print(f"S: real loss: {ds_loss_real}, fake loss: {ds_loss_fake}")
    print(f"T: real loss: {dt_loss_real}, fake loss: {dt_loss_fake}")
    if it%100 == 0 and it>0:
        src.sample_images(it, gan_test[...,1:], gan_test_truth, past, generator)
        src.plot_temporal_training_curves(log, it, name)

src.sample_images(iterations, gan_test[...,1:], gan_test_truth, past, generator)
src.plot_temporal_training_curves(log, iterations, name)

## Save features

Save model history

In [ ]:
np.save(name+"_log",log)

Save model weights

In [ ]:
combined.save_weights(name+"_model.h5")

Load model and predict

In [ ]:
combined.load_weights("C:/Users/pkicsiny/Desktop/TUM/3/ADL4CV/ADL4CV_project/"+name+"_model.h5")

Predict future frames. Loads a 20 long sequence with 1000 sequence samples.

In [19]:
sequence_test = src.load_datasets(prediction=True)

Test data: (1000, 64, 64, 20)


In [20]:
sequence_test = src.augment_data(sequence_test[:100])

In [21]:
generator = combined.layers[1]

In [ ]:
#test
predictions = {}
past_frames = sequence_test[...,0:past]
test_truth = sequence_test[...,past:past+1]
for t in range(5):
    print(t)
    future = generator.predict(past_frames, batch_size=64)
    predictions[f"{t}"] = future
    past_frames = np.concatenate((past_frames[:,:,:,1:], predictions[f"{t}"]), axis=-1)
    test_truth = sequence_test[...,past+1+t:past+2+t]

0


Save example predictions

In [ ]:
src.save_examples(name, sequence_test, predictions, past, samples=[33,46,54])

Renormalize intensity values

In [ ]:
norms = np.load(sys.path[0]+"/5min_norms_compressed.npz")["arr_1"]

In [ ]:
# *4 bc of augmentaion (it concats the frames so the 0th 1000th 2000th and 3000th are the same sample just rotated)
test_norms = list(norms[9000:])*4

In [ ]:
#renormalize test samples
renormalized_test = np.array([sample * np.array(test_norms)[i] for i, sample in enumerate(sequence_test)])
renormalized_predictions = np.transpose((np.array([[sample * np.array(test_norms)[i] for i, sample in enumerate(predictions[key])] for key in ['0', '1', '2', '3', "4"]])[:,:,:,:,0]), (1,2,3,0))

Calculate pixel intensities back to dBZ and from there to mm/h. <br>
Sources: <br>
- https://www.dwd.de/DE/leistungen/radolan/radolan_info/radolan_radvor_op_komposit_format_pdf.pdf?__blob=publicationFile&v=11 (page 10)
- <https://web.archive.org/web/20160113151652/http://www.desktopdoppler.com/help/nws-nexrad.htm#rainfall%20rates>

In [ ]:
#dBZ
dBZ_t = renormalized_test*0.5 - 32.5
dBZ_p = renormalized_predictions*0.5 - 32.5
#mm/h
I_t = (0.005*10**(0.1*dBZ_t))**(0.625)
I_p = (0.005*10**(0.1*dBZ_p))**(0.625)

Calculate scores

In [ ]:
thresholds = [0.5]
scores = {}
for t in range(5): # loop over the predictions (4)
    for s in thresholds: # make a dict entry for each threshold score
        scores[f"pred_{t+1}"] = src.calculate_skill_scores(I_p[...,t:t+1],
                                                          I_t[...,past+t:past+1+t],
                                                          x=I_t[...,past-1:past],
                                                          threshold=s)

In [ ]:
np.save(name+"_scores",scores)